In [273]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import random

In [277]:
# LOOKING AT THE DATA
spy = yf.download("SPY", start="1993-01-01", end="2025-06-30", auto_adjust=False, progress=False)
spy = spy.reset_index()
# spy.columns = [col[0] if isinstance(col, tuple) else col for col in spy.columns]
spy.columns = spy.columns.get_level_values(0)  # flatten in case of multiindex

print(spy.head())
print(spy.dtypes)


Price       Date  Adj Close     Close      High       Low      Open   Volume
0     1993-01-29  24.380451  43.93750  43.96875  43.75000  43.96875  1003200
1     1993-02-01  24.553852  44.25000  44.25000  43.96875  43.96875   480500
2     1993-02-02  24.605869  44.34375  44.37500  44.12500  44.21875   201300
3     1993-02-03  24.865973  44.81250  44.84375  44.37500  44.40625   529400
4     1993-02-04  24.970009  45.00000  45.09375  44.46875  44.96875   531500
Price
Date         datetime64[ns]
Adj Close           float64
Close               float64
High                float64
Low                 float64
Open                float64
Volume                int64
dtype: object


In [278]:
# RANDOM 100 SIMULATION ON INVESTMENT ENTRY FOR 5 YEARS (ONLY DATA FROM 2006- BECAUSE OF SSO)
# spy = yf.download("SPY", start="1993-01-01", end="2024-12-31", auto_adjust=False, progress=False) #EARLIEST DATA AVAILABLE
sso = yf.download("SSO", start="1993-01-01", end="2025-06-30", auto_adjust=False, progress=False)

# Reset index so Date becomes a column
# spy = spy.reset_index()
sso = sso.reset_index()

# Common dates
common_dates = pd.Series(np.intersect1d(spy['Date'].dt.date, sso['Date'].dt.date))
common_dates = common_dates.sort_values().reset_index(drop=True)

# Function to compute returns for a 5-year period
def compute_return(start_date, end_date):
    open_spy = spy.loc[spy['Date'].dt.date == start_date, 'Open'].values[0]
    close_spy = spy.loc[spy['Date'].dt.date == end_date, 'Close'].values[0]
    spy_return = close_spy / open_spy

    open_sso = sso.loc[sso['Date'].dt.date == start_date, 'Open'].values[0]
    close_sso = sso.loc[sso['Date'].dt.date == end_date, 'Close'].values[0]
    sso_return = close_sso / open_sso

    return start_date, end_date, spy_return, sso_return

# Simulate 100 random 5-year periods
results = []
for _ in range(100):
    while True:
        idx = random.randint(0, len(common_dates) - 1 - 1260)  # ~252 trading days/year * 5 years
        start_date = common_dates[idx]
        end_date = common_dates[idx + 1260]

        try:
            result = compute_return(start_date, end_date)
            results.append(result)
            break
        except IndexError:
            continue

# Print results
df = pd.DataFrame(results, columns=["Start Date", "End Date", "SPY Return", "SSO Return"])
print(df)


    Start Date    End Date  SPY Return            SSO Return
0   2009-12-21  2014-12-23    1.875677   [3.470105841704391]
1   2009-05-20  2014-05-22    2.061881   [4.139347600350337]
2   2010-07-29  2015-07-31    1.887554  [3.5809370362708024]
3   2013-05-07  2018-05-08    1.646333  [2.7553165328965115]
4   2018-09-13  2023-09-18    1.528072  [1.8265273974116332]
..         ...         ...         ...                   ...
95  2015-04-22  2020-04-23    1.328889  [1.5404757463346828]
96  2007-02-02  2012-02-02    0.916741   [0.577529364001027]
97  2019-09-12  2024-09-16    1.868349   [2.637228609060665]
98  2010-07-23  2015-07-27    1.892988   [3.601664329701593]
99  2006-09-01  2011-09-02    0.898658  [0.5319093870885164]

[100 rows x 4 columns]


In [279]:
#PRINT TOTAL RETURNS FOR ALL 100 INVESTMENTS
df['spy_return'] = df['SPY Return'] * 1000
df['sso_return'] = df['SSO Return'] * 1000
print(sum(df['spy_return']))
print(sum(df['sso_return']))


166122.61730657084
[256325.79059243]


In [281]:
#CREATING SYNTHETIC SSO FOR EARLIER DATA

spy_to_create = yf.download("SPY", start="1993-01-01", end="2025-07-01", auto_adjust=False, progress=False)

# Calculate daily price returns using Close
spy_to_create['Daily Return'] = spy_to_create['Close'].pct_change()

# Simulate synthetic SSO: 2x SPY daily returns
spy_to_create['Synthetic SSO Return'] = 2 * spy_to_create['Daily Return']
spy_to_create['Synthetic SSO Price'] = (1 + spy_to_create['Synthetic SSO Return']).cumprod()

# Create synthetic_sso DataFrame
synthetic_sso = spy_to_create[['Synthetic SSO Return', 'Synthetic SSO Price']].copy()
synthetic_sso.rename(columns={
    'Synthetic SSO Return': 'Return',
    'Synthetic SSO Price': 'Price'
}, inplace=True)
synthetic_sso = synthetic_sso.reset_index()
synthetic_sso.columns = synthetic_sso.columns.get_level_values(0)
synthetic_sso = synthetic_sso.iloc[1:]

print(synthetic_sso.head())


Price       Date    Return     Price
1     1993-02-01  0.014225  1.014225
2     1993-02-02  0.004237  1.018522
3     1993-02-03  0.021142  1.040056
4     1993-02-04  0.008368  1.048759
5     1993-02-05 -0.001389  1.047302


In [282]:
#SYNTHETIC_SSO TO GET DATA FROM 1993 W 100 SIM, 5 YEAR
############################################################################################################
# Download data

# Make sure your synthetic_sso DataFrame exists and has columns: ['Date', 'Synthetic_SSO']
# Convert 'Date' columns to datetime
spy['Date'] = pd.to_datetime(spy['Date'])
synthetic_sso['Date'] = pd.to_datetime(synthetic_sso['Date'])

# Same trading days
common_dates = pd.Series(np.intersect1d(spy['Date'].dt.date, synthetic_sso['Date'].dt.date))
common_dates = common_dates.sort_values().reset_index(drop=True)

# Function to compute returns for a 5-year period
def compute_return(start_date, end_date):
    # SPY return: Open to Close
    open_spy = spy.loc[spy['Date'].dt.date == start_date, 'Open'].values[0]
    close_spy = spy.loc[spy['Date'].dt.date == end_date, 'Close'].values[0]
    spy_return = close_spy / open_spy

    # Synthetic SSO return: value at start vs value at end
    start_sso = synthetic_sso.loc[synthetic_sso['Date'].dt.date == start_date, 'Price'].values[0]
    end_sso = synthetic_sso.loc[synthetic_sso['Date'].dt.date == end_date, 'Price'].values[0]
    sso_return = end_sso / start_sso

    return start_date, end_date, spy_return, sso_return

# Simulate 100 random 5-year periods
results = []
for _ in range(100):
    while True:
        idx = random.randint(0, len(common_dates) - 1 - 1260)
        start_date = common_dates[idx]
        end_date = common_dates[idx + 1260]

        try:
            result = compute_return(start_date, end_date)
            results.append(result)
            break
        except IndexError:
            continue

# DF of results w print
df = pd.DataFrame(results, columns=["Start Date", "End Date", "SPY Return", "Synthetic SSO Return"])
print(df.head())


   Start Date    End Date  SPY Return  Synthetic SSO Return
0  2020-02-10  2025-02-13    1.840805              2.646488
1  2007-03-09  2012-03-08    0.969783              0.666748
2  2002-05-15  2007-05-17    1.381735              1.681452
3  2011-10-12  2016-10-14    1.767164              2.830517
4  1995-08-31  2000-08-28    2.693566              6.037230


In [283]:
#PRINT TOTAL RETURNS FOR ALL 100 INVESTMENTS
df['spy_return'] = df['SPY Return'] * 1000
df['sso_return'] = df['Synthetic SSO Return'] * 1000
print(sum(df['spy_return']))
print(sum(df['sso_return']))


156522.0643777378
244851.5684142639


In [284]:
#SYNTHETIC_SSO TO GET DATA FROM 1993 INVESTMENT EVERY 2 MONTHS HOLD FOR 5 YEARS
############################################################################################################

# Convert date columns to datetime
spy['Date'] = pd.to_datetime(spy['Date'])
synthetic_sso['Date'] = pd.to_datetime(synthetic_sso['Date'])

# Same Trading Days
common_dates = pd.Series(np.intersect1d(spy['Date'].dt.date, synthetic_sso['Date'].dt.date))
common_dates = common_dates.sort_values().reset_index(drop=True)

# Function to compute returns for a 5-year period
def compute_return(start_date, end_date):
    # SPY return: Open to Close
    open_spy = spy.loc[spy['Date'].dt.date == start_date, 'Open'].values[0]
    close_spy = spy.loc[spy['Date'].dt.date == end_date, 'Close'].values[0]
    spy_return = close_spy / open_spy

    # Synthetic SSO return: synthetic value change
    start_sso = synthetic_sso.loc[synthetic_sso['Date'].dt.date == start_date, 'Price'].values[0]
    end_sso = synthetic_sso.loc[synthetic_sso['Date'].dt.date == end_date, 'Price'].values[0]
    sso_return = end_sso / start_sso

    return start_date, end_date, spy_return, sso_return

# Simulation using a new start every ~2 months (roughly every 40 trading days)
step = 40  # about 2 months of trading days
window = 1260  # ~5 years
results = []

for idx in range(0, len(common_dates) - window, step):
    start_date = common_dates[idx]
    end_date = common_dates[idx + window]
    
    try:
        result = compute_return(start_date, end_date)
        results.append(result)
    except IndexError:
        continue

# DF + results display
df = pd.DataFrame(results, columns=["Start Date", "End Date", "SPY Return", "Synthetic SSO Return"])
df['spy_return'] = df['SPY Return'] * 1000
df['sso_return'] = df['Synthetic SSO Return'] * 1000
print(df.head())
print(f"Total SPY final value across all simulations: ${sum(df['spy_return'])}")
print(f"Total SSO final value across all simulations: ${sum(df['sso_return'])}")


   Start Date    End Date  SPY Return  Synthetic SSO Return   spy_return  \
0  1993-02-01  1998-01-27    2.202559              4.384259  2202.558635   
1  1993-03-30  1998-03-25    2.439446              5.432856  2439.446367   
2  1993-05-26  1998-05-21    2.473356              5.489144  2473.356401   
3  1993-07-23  1998-07-20    2.658725              6.411223  2658.724597   
4  1993-09-20  1998-09-15    2.268392              4.689909  2268.392371   

    sso_return  
0  4384.259212  
1  5432.855888  
2  5489.144118  
3  6411.222890  
4  4689.909430  
Total SPY final value across all simulations: $267912.00225253915
Total SSO final value across all simulations: $399804.3339956073


In [285]:
#SYNTHETIC_SSO TO GET DATA FROM 1993 RECURRENT INVESTMENT EVERY WEEK FOR 5 YEARS
############################################################################################################
# 1. Download SPY data

# 2. Ensure synthetic_sso exists and is clean
synthetic_sso['Date'] = pd.to_datetime(synthetic_sso['Date'])
synthetic_sso = synthetic_sso[['Date', 'Price']].dropna()

# 3. Merge SPY and SSO on Date
merged = pd.merge(spy[['Date', 'Close']], synthetic_sso, on='Date', how='inner')
merged = merged.sort_values('Date').reset_index(drop=True)
print(merged.head())

# 4. Find January start dates at least 5 years before end of data
# Get the first trading day in January for each year
jan_starts = (
    merged[merged['Date'].dt.month == 1]
    .groupby(merged['Date'].dt.year)['Date']
    .min()
    .reset_index(drop=True)
)
jan_starts = jan_starts[jan_starts <= merged['Date'].max() - pd.DateOffset(years=5)]
# 5. Simulate monthly $100 investments over 5 years
results = []
for start_date in jan_starts:
    end_date = start_date + pd.DateOffset(years=5)
        # Get trading dates only
    all_trading_dates = merged['Date']
    
    # Build investment dates: first trading day of each month
    monthly_dates = (
        all_trading_dates[(all_trading_dates >= start_date) & (all_trading_dates <= end_date)]
        .groupby(all_trading_dates.dt.to_period("M"))
        .first()
        .values
    )


    invest_subset = merged[merged['Date'].isin(monthly_dates)].copy()
    if len(invest_subset) < 55:
        continue

    spy_units = (100 / invest_subset['Close']).sum()
    sso_units = (100 / invest_subset['Price']).sum()

    final_date = invest_subset['Date'].iloc[-1]
    final_row = merged[merged['Date'] == final_date]
    if final_row.empty:
        continue

    spy_final = spy_units * final_row['Close'].values[0]
    sso_final = sso_units * final_row['Price'].values[0]

    results.append({
        'Start Date': start_date,
        'End Date': final_date,
        'SPY Final Value': round(spy_final, 2),
        'SSO Final Value': round(sso_final, 2)
    })
# 6. DF + Display Results
df = pd.DataFrame(results)
print(df.head())
print(f"Total SPY final value across all simulations: ${sum(df['SPY Final Value'])}")
print(f"Total SSO final value across all simulations: ${sum(df['SSO Final Value'])}")

Price       Date     Close     Price
0     1993-02-01  44.25000  1.014225
1     1993-02-02  44.34375  1.018522
2     1993-02-03  44.81250  1.040056
3     1993-02-04  45.00000  1.048759
4     1993-02-05  44.96875  1.047302
  Start Date   End Date  SPY Final Value  SSO Final Value
0 1994-01-03 1998-12-01         10804.45         19208.31
1 1995-01-03 2000-01-03         10997.13         19594.89
2 1996-01-02 2001-01-02          7912.99          9665.31
3 1997-01-02 2002-01-02          6188.93          5695.98
4 1998-01-02 2003-01-02          4718.33          3284.23
Total SPY final value across all simulations: $205300.59
Total SSO final value across all simulations: $253651.26


In [286]:
#DIVIDEND + SYNTHETIC_SSO TO GET DATA FROM 1993 RECURRENT INVESTMENT EVERY WEEK FOR 5 YEARS 
############################################################################################################
#Ensure synthetic_sso exists and is clean
synthetic_sso['Date'] = pd.to_datetime(synthetic_sso['Date'])
synthetic_sso = synthetic_sso[['Date', 'Price']].dropna()

#Merge SPY and SSO on Date
merged = pd.merge(spy[['Date', 'Close', 'Adj Close']], synthetic_sso, on='Date', how='inner')
merged = merged.sort_values('Date').reset_index(drop=True)
print(merged.head())

#Find January start dates at least 5 years before end of data
# Get the first trading day in January for each year
jan_starts = (
    merged[merged['Date'].dt.month == 1]
    .groupby(merged['Date'].dt.year)['Date']
    .min()
    .reset_index(drop=True)
)
jan_starts = jan_starts[jan_starts <= merged['Date'].max() - pd.DateOffset(years=5)]

#Simulate monthly $100 investments over 5 years
results = []
for start_date in jan_starts:
    end_date = start_date + pd.DateOffset(years=5)
        # Get trading dates only
    all_trading_dates = merged['Date']
    
    # Build investment dates: first trading day of each month
    monthly_dates = (
        all_trading_dates[(all_trading_dates >= start_date) & (all_trading_dates <= end_date)]
        .groupby(all_trading_dates.dt.to_period("M"))
        .first()
        .values
    )


    invest_subset = merged[merged['Date'].isin(monthly_dates)].copy()
    if len(invest_subset) < 55:
        continue

    spy_units = (100 / invest_subset['Close']).sum()
    spy_adj_units = (100 / invest_subset['Adj Close']).sum()
    sso_units = (100 / invest_subset['Price']).sum()

    final_date = invest_subset['Date'].iloc[-1]
    final_row = merged[merged['Date'] == final_date]
    if final_row.empty:
        continue

    spy_final = spy_units * final_row['Close'].values[0]
    spy__adj_final = spy_adj_units * final_row['Adj Close'].values[0]
    sso_final = sso_units * final_row['Price'].values[0]

    results.append({
        'Start Date': start_date,
        'End Date': final_date,
        'SPY Final Value': round(spy_final, 2),
        'SPY ADJ Final Value': round(spy__adj_final, 2),
        'SSO Final Value': round(sso_final, 2)
    })
#DF + Display Results
df = pd.DataFrame(results)
print(df.head(5))
print(f"Total SPY (adjusted) final value across all simulations: ${sum(df['SPY ADJ Final Value'])}")
print(f"Total SSO final value across all simulations: ${sum(df['SSO Final Value'])}")

Price       Date     Close  Adj Close     Price
0     1993-02-01  44.25000  24.553852  1.014225
1     1993-02-02  44.34375  24.605869  1.018522
2     1993-02-03  44.81250  24.865973  1.040056
3     1993-02-04  45.00000  24.970009  1.048759
4     1993-02-05  44.96875  24.952669  1.047302
  Start Date   End Date  SPY Final Value  SPY ADJ Final Value  SSO Final Value
0 1994-01-03 1998-12-01         10804.45             11358.98         19208.31
1 1995-01-03 2000-01-03         10997.13             11497.03         19594.89
2 1996-01-02 2001-01-02          7912.99              8213.60          9665.31
3 1997-01-02 2002-01-02          6188.93              6399.67          5695.98
4 1998-01-02 2003-01-02          4718.33              4878.69          3284.23
Total SPY (adjusted) final value across all simulations: $215598.24
Total SSO final value across all simulations: $253651.26


In [287]:
#DIVIDEND + SYNTHETIC_SSO TO GET DATA FROM 1993 RECURRENT INVESTMENT EVERY WEEK FOR 5 YEARS 
############################################################################################################

#Ensure synthetic_sso exists and is clean
synthetic_sso['Date'] = pd.to_datetime(synthetic_sso['Date'])
synthetic_sso = synthetic_sso[['Date', 'Price']].dropna()

#Merge SPY and SSO on Date
merged = pd.merge(spy[['Date', 'Close', 'Adj Close']], synthetic_sso, on='Date', how='inner')
merged = merged.sort_values('Date').reset_index(drop=True)
print(merged.head())

#Find January start dates at least 5 years before end of data
# Get the first trading day in January for each year
jan_starts = (
    merged[merged['Date'].dt.month == 1]
    .groupby(merged['Date'].dt.year)['Date']
    .min()
    .reset_index(drop=True)
)
jan_starts = jan_starts[jan_starts <= merged['Date'].max() - pd.DateOffset(years=5)]

#Simulate monthly $100 investments over 5 years
results = []
for start_date in jan_starts:
    end_date = start_date + pd.DateOffset(years=5)
        # Get trading dates only
    all_trading_dates = merged['Date']
    
    # Build investment dates: first trading day of each month
    monthly_dates = (
        all_trading_dates[(all_trading_dates >= start_date) & (all_trading_dates <= end_date)]
        .groupby(all_trading_dates.dt.to_period("M"))
        .first()
        .values
    )


    invest_subset = merged[merged['Date'].isin(monthly_dates)].copy()
    if len(invest_subset) < 55:
        continue

    spy_units = (100 / invest_subset['Close']).sum()
    spy_adj_units = (100 / invest_subset['Adj Close']).sum()
    sso_units = (100 / invest_subset['Price']).sum()

    final_date = invest_subset['Date'].iloc[-1]
    final_row = merged[merged['Date'] == final_date]
    if final_row.empty:
        continue

    spy_final = spy_units * final_row['Close'].values[0]
    spy__adj_final = spy_adj_units * final_row['Adj Close'].values[0]
    sso_final = sso_units * final_row['Price'].values[0]

    results.append({
        'Start Date': start_date,
        'End Date': final_date,
        'SPY Final Value': round(spy_final, 2),
        'SPY ADJ Final Value': round(spy__adj_final, 2),
        'SSO Final Value': round(sso_final, 2)
    })
#DF + Display Results
df = pd.DataFrame(results)
print(df.head(5))
print(f"Total SPY (adjusted) final value across all simulations: ${sum(df['SPY ADJ Final Value'])}")
print(f"Total SSO final value across all simulations: ${sum(df['SSO Final Value'])}")

Price       Date     Close  Adj Close     Price
0     1993-02-01  44.25000  24.553852  1.014225
1     1993-02-02  44.34375  24.605869  1.018522
2     1993-02-03  44.81250  24.865973  1.040056
3     1993-02-04  45.00000  24.970009  1.048759
4     1993-02-05  44.96875  24.952669  1.047302
  Start Date   End Date  SPY Final Value  SPY ADJ Final Value  SSO Final Value
0 1994-01-03 1998-12-01         10804.45             11358.98         19208.31
1 1995-01-03 2000-01-03         10997.13             11497.03         19594.89
2 1996-01-02 2001-01-02          7912.99              8213.60          9665.31
3 1997-01-02 2002-01-02          6188.93              6399.67          5695.98
4 1998-01-02 2003-01-02          4718.33              4878.69          3284.23
Total SPY (adjusted) final value across all simulations: $215598.24
Total SSO final value across all simulations: $253651.26


In [288]:
#100 DOLLAR INVESTMENT EACH WEEK FOR DIFFERENT LEVERAGE MULTIPLIERS
############################################################################################################

#Funciton for synthetic returns for different leverage multipliers
def create_synthetic_leverage_df(spy_df, leverage):
    df = spy_df.copy()
    df['Daily Return'] = df['Close'].pct_change()
    df[f'{leverage}x Return'] = leverage * df['Daily Return']
    df[f'{leverage}x Price'] = (1 + df[f'{leverage}x Return']).cumprod()
    synthetic = df[['Date', f'{leverage}x Price']].dropna()
    synthetic.rename(columns={f'{leverage}x Price': 'Price'}, inplace=True)
    return synthetic

# Leverage levels to test
leverage_levels = [1, 2, 3, 4, 5]

# Store results
all_results = []

#Loop through leverage multipliers
for lev in leverage_levels:
    if lev == 1:
        synthetic = spy[['Date', 'Close']].copy()
        synthetic.rename(columns={'Close': 'Price'}, inplace=True)
    else:
        synthetic = create_synthetic_leverage_df(spy, lev)
        synthetic.columns = synthetic.columns.get_level_values(0)

    # Merge with SPY base
    merged = pd.merge(spy[['Date', 'Close', 'Adj Close']], synthetic, on='Date', how='inner')
    
    merged = merged.sort_values('Date').reset_index(drop=True)

    # Find January start dates at least 5 years before end
    jan_starts = (
        merged[merged['Date'].dt.month == 1]
        .groupby(merged['Date'].dt.year)['Date']
        .min()
        .reset_index(drop=True)
    )
    jan_starts = jan_starts[jan_starts <= merged['Date'].max() - pd.DateOffset(years=5)]

    for start_date in jan_starts:
        end_date = start_date + pd.DateOffset(years=5)

        all_trading_dates = merged['Date']
        monthly_dates = (
            all_trading_dates[(all_trading_dates >= start_date) & (all_trading_dates <= end_date)]
            .groupby(all_trading_dates.dt.to_period("M"))
            .first()
            .values
        )

        invest_subset = merged[merged['Date'].isin(monthly_dates)].copy()
        if len(invest_subset) < 55:
            continue

        spy_units = (100 / invest_subset['Close']).sum()
        spy_adj_units = (100 / invest_subset['Adj Close']).sum()
        lev_units = (100 / invest_subset['Price']).sum()

        final_date = invest_subset['Date'].iloc[-1]
        final_row = merged[merged['Date'] == final_date]
        if final_row.empty:
            continue

        spy_final = spy_units * final_row['Close'].values[0]
        spy_adj_final = spy_adj_units * final_row['Adj Close'].values[0]
        lev_final = lev_units * final_row['Price'].values[0]

        all_results.append({
            'Leverage': f'{lev}x',
            'Start Date': start_date,
            'End Date': final_date,
            'SPY Final Value': round(spy_final, 2),
            'SPY ADJ Final Value': round(spy_adj_final, 2),
            f'{lev}x Final Value': round(lev_final, 2)
        })

df = pd.DataFrame(all_results)

# Pivot for comparison
pivot = df.pivot_table(index='Start Date', columns='Leverage', values=[f'{lev}x Final Value' for lev in leverage_levels if lev > 1])
print(pivot.head())
print(pivot.sum())



           2x Final Value 3x Final Value 4x Final Value 5x Final Value
Leverage               2x             3x             4x             5x
Start Date                                                            
1994-01-03       19208.31       32595.77       51446.90       74063.91
1995-01-03       19594.89       33089.97       51308.98       71381.45
1996-01-02        9665.31       10677.93       10349.38        8617.77
1997-01-02        5695.98        4682.35        3442.87        2330.89
1998-01-02        3284.23        2137.86        1388.26         961.11
                Leverage
2x Final Value  2x          253651.26
3x Final Value  3x          309211.24
4x Final Value  4x          368408.37
5x Final Value  5x          423475.88
dtype: float64


In [289]:
#5 YEAR INVESTMENT TESTED FOR EACH 2 MONTH START-DATE FOR DIFFERENT LEVERAGE MULTIPLIERS
############################################################################################################
step = 40       # every 2 months (~40 trading days)
window = 1260   # ~5 years

# Store results
all_results = []

for lev in leverage_levels:
    # Create synthetic ETF (or just use SPY for 1x)
    if lev == 1:
        synthetic = spy[['Date', 'Close']].copy()
        synthetic.columns = synthetic.columns.get_level_values(0)
        synthetic.rename(columns={'Close': 'Price'}, inplace=True)
    else:
        synthetic = create_synthetic_leverage_df(spy, lev)
        synthetic.columns = synthetic.columns.get_level_values(0)
    
    # Merge with SPY data for consistent date alignment
    spy.columns = spy.columns.get_level_values(0) 
    merged = pd.merge(spy[['Date', 'Open', 'Close']], synthetic, on='Date', how='inner')
    merged = merged.sort_values('Date').reset_index(drop=True)

    # Create list of trading days
    trading_dates = merged['Date'].dt.date.values
    for idx in range(0, len(trading_dates) - window, step):
        start_date = trading_dates[idx]
        end_date = trading_dates[idx + window]

        try:
            # Get SPY open and close for the window
            open_spy = merged.loc[merged['Date'].dt.date == start_date, 'Open'].values[0]
            close_spy = merged.loc[merged['Date'].dt.date == end_date, 'Close'].values[0]
            spy_return = close_spy / open_spy

            # Get leveraged ETF return for the window
            start_lev = merged.loc[merged['Date'].dt.date == start_date, 'Price'].values[0]
            end_lev = merged.loc[merged['Date'].dt.date == end_date, 'Price'].values[0]
            lev_return = end_lev / start_lev

            all_results.append({
                'Leverage': f'{lev}x',
                'Start Date': start_date,
                'End Date': end_date,
                'SPY Return': spy_return,
                f'{lev}x Return': lev_return,
                'SPY Value': spy_return * 100,
                f'{lev}x Value': lev_return * 100
            })
        except (IndexError, ValueError):
            continue
df = pd.DataFrame(all_results)
# Optional: pivot table to compare across leverage levels
pivot = df.pivot_table(index='Start Date', columns='Leverage', values=[f'{lev}x Value' for lev in leverage_levels if lev > 1])
pivot.columns = pivot.columns.droplevel(0)  # Flatten
print(pivot.head())

# Sum final values for each strategy
print("\nTotal value by leverage level:")
print(pivot.sum())


Leverage            2x           3x           4x           5x
Start Date                                                   
1993-02-01  438.425921   803.139475  1343.371150  2047.302945
1993-03-30  543.285589  1108.180028  2064.486311  3505.314780
1993-05-26  548.914412  1124.133113  2100.959713  3576.041605
1993-07-23  641.122289  1412.692920  2832.375127  5156.293277
1993-09-20  468.990943   857.013410  1393.395515  2007.362217

Total value by leverage level:
Leverage
2x     39980.433400
3x     57816.733670
4x     80170.280894
5x    104884.587391
dtype: float64


In [290]:
# INVESTMENT PERFORMANCE STARETD IN 1993 HELD OVER DIFFERENT LEVERAGE MULTIPLIERS(FINDING MAX EV MULITPLIER)
############################################################################################################

# Leverage levels to test
leverage_levels_detailed = [1, 2, 2.6, 2.7, 2.8, 2.85, 2.9, 3, 3.1, 4, 5]

# Results list
results = []

for lev in leverage_levels_detailed:
    if lev == 1:
        synthetic = spy[['Date', 'Close']].copy()
        synthetic.rename(columns={'Close': 'Price'}, inplace=True)
    else:
        synthetic = create_synthetic_leverage_df(spy, lev)
        synthetic.columns = synthetic.columns.get_level_values(0)

    synthetic = synthetic.sort_values('Date').reset_index(drop=True)

    # Starting and ending prices
    start_price = synthetic.loc[synthetic['Date'].dt.year == 1993, 'Price'].iloc[0]
    end_price = synthetic.iloc[-1]['Price']

    value = (100 / start_price) * end_price  # $100 initial investment
    results.append({'Leverage': f'{lev}x', 'Start Price': round(start_price, 2), 'End Price': round(end_price, 2), 'Final Value': round(value, 2)})

# Show results
df = pd.DataFrame(results)
print(df)


   Leverage  Start Price  End Price  Final Value
0        1x        43.94     614.91      1399.51
1        2x         1.01      62.21      6133.60
2      2.6x         1.02      87.19      8560.48
3      2.7x         1.02      88.51      8684.48
4      2.8x         1.02      88.80      8706.44
5     2.85x         1.02      88.55      8678.66
6      2.9x         1.02      88.03      8625.24
7        3x         1.02      86.24      8443.42
8      3.1x         1.02      83.47      8167.03
9        4x         1.03      36.02      3502.82
10       5x         1.04       4.29       414.27
